# Habituation with IBCM and BioPCA to turbulent backgrounds with non-linear ORN model
The details of the model are described in other Jupyter notebooks (e.g., ibcm_gating_linear_multiple_components.ipynb). 

Importantly, the background used is an approximation of realistic odor concentrations in turbulent environments, based on Celani, Villermaux, Vergassola, 2014. In summary, each odor concentration (sources treated as independent for now) is a succession of whiffs and blanks, with power law distributions of exponent $-3/2$ for the waiting times, and a long-tailed whiff concentration distribution of the form $\sim e^{-c/c_0}/c$. 

Also, of note, we use the Euclidean distance between $\vec{s}$ (the filtered odor, projection neurons layer) and the target odor as a performance metric, instead of the Jaccard similarity of the Kenyon cells layer. The reason: it is easier to work with analytically, easier to understand intuitively, and easier to compute numerically. Anyways, since KC cells implement a locality sensitive hashing, closeness in Euclidean distance should correspond to closeness in neural tags (hashes). 

Lastly, odors are now encoded by two constant vectors: binding affinities $\vec{\kappa}$ and activation affinities $\vec{\eta}$, and they are combined according to the ORN model from Reddy et al., 2018, equations 16-17. So, if we have odors $\gamma = 1, 2, \ldots, K$ with concentrations $\nu_{\gamma}$ and vectors $(\vec{\kappa}_{\gamma}, \vec{\eta}_{\gamma})$, the total activity of ORN $j = 1, 2, \ldots, N$ is:

$$ F_i(\vec{\nu}) = \frac{F_{max}}{1 + \left(\frac{1 + C/\kappa_{j, mix}}{\chi_{j, M} \eta_{j, mix}C/\kappa_{j, mix}} \right)^n }$$ 

where

 - $ C = \sum_{\gamma=1}^K \nu_{\gamma}$ is the total concentration
 - $n = 4$ is the number of activation steps of ionic CNG channels
 - $\kappa_{j, mix}^{-1} = \sum_{\gamma=1}^K \frac{\beta_{\gamma}}{\kappa_{j, \gamma}} $ is the mixture's binding affinity for receptor type $j$.
 - $\eta_{j, mix} = \kappa_{j, mix} \sum_{\gamma=1}^K \eta_{j, \gamma} \beta_{\gamma} / \kappa_{j, \gamma}$ is the mixture's activation affinity for receptor type $j$.  
 - $\beta_{\gamma} = \nu_{\gamma} / C$ is the fraction of odor $\gamma$ in the total concentration 
 - $\chi_{j, M} = 1 - \sum_{\gamma=1}^K \nu_{\gamma} \tilde{M}_{j, \gamma}$ is the masking coefficient
 - $\tilde{M}_{\gamma} = \frac{K_{M, j, \gamma}\nu_{\gamma}}{1 + \sum_{\rho=1}^K K_{M, j, \rho} \nu_{\rho}}$ is the masking concentration of odor $\gamma$ at olfactory receptor type $j$.
 - $\mu_{j, \gamma}$ is the masking efficiency, a number between $0$ and $1$. 
 - $F_{max}$ is the maximum activation of an ORN, which we set to 1. 
 
For the moment, we set the masking coefficient $\chi_{j, M} = 1$, i.e. there is no masking. 


## Functions of general interest

In [ ]:
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from time import perf_counter
import os, json
import sys
if "../" not in sys.path:
    sys.path.insert(1, "../")

from modelfcts.ibcm import (
    integrate_inhib_ibcm_network_options,
    ibcm_respond_new_odors,
    compute_mbars_cgammas_cbargammas,
    ibcm_respond_new_odors
)
from modelfcts.ibcm_analytics import (
    fixedpoint_thirdmoment_exact, 
    fixedpoint_thirdmoment_perturbtheory,
    ibcm_fixedpoint_w_thirdmoment, 
    ibcm_all_largest_eigenvalues
)
from modelfcts.biopca import (
    integrate_inhib_ifpsp_network_skip,
    build_lambda_matrix,
    biopca_respond_new_odors
)
from modelfcts.average_sub import (
    integrate_inhib_average_sub_skip, 
    average_sub_respond_new_odors
)
from modelfcts.ideal import (
    find_projector, 
    find_parallel_component, 
    ideal_linear_inhibitor, 
    compute_optimal_factor
)
from modelfcts.checktools import (
    check_conc_samples_powerlaw_exp1,
    compute_pca_meankept, 
    compute_projector_series, 
    analyze_pca_learning
)
from modelfcts.backgrounds import (
    update_powerlaw_times_concs, 
    logof10, 
    sample_background_powerlaw,
    sample_ss_conc_powerlaw, 
    decompose_nonorthogonal_basis, 
    update_alternating_inputs, 
    generate_odorant, 
    update_tc_odor
)
from modelfcts.tagging import (
    project_neural_tag, 
    create_sparse_proj_mat, 
    SparseNDArray, 
    tags_list_to_csr_matrix
)
from utils.statistics import seed_from_gen
from modelfcts.distribs import (
    truncexp1_inverse_transform, 
    truncexp1_density, 
    truncexp1_average,
    powerlaw_cutoff_inverse_transform
)
from utils.smoothing_function import (
    moving_average, 
    moving_var
)
from simulfcts.plotting import (
    plot_cbars_gammas_sums, 
    plot_cbars_gamma_series, 
    plot_3d_series, 
    plot_w_matrix, 
    plot_background_norm_inhibition, 
    plot_background_neurons_inhibition, 
    plot_pca_results, 
    hist_outline
)
from simulfcts.analysis import compute_back_reduction_stats
from utils.metrics import jaccard
from modelfcts.orn_model import (
    generate_odorant_kappaeta, 
    combine_odors_ornmodel
)

## Main new function: background update with ORN model

In [ ]:
# Function to update the background by combining odorant at the current concentrations with the ORN model
def update_powerlaw_times_concs_ornmodel(tc_bk, params_bk, noises, dt):
    """
    Simulate turbulent odors by pulling wait times until the end of a whiff
        or until the next blank, and a concentration of the whiff.
        For each odor, check whether the time left until switch is <= zero;
        if so, pull either
            - another wait time t_w if current c=0, and pull the new c > 0
              (we were in a blank and are starting a whiff)
            - another wait time t_b if current c > 0, and set c = 0
              (we were in a whiff and are starting a blank)
        Otherwise, decrement t by dt and don't change c.

    Args:
        tc_bk (np.ndarray): array of t, c for each odor in the background,
            where t = time left until next change, c = current concentration
            of the odor. Shaped [n_odors, 2]
        params_bk (list): contains the following elements (a lot needed!):
            whiff_tmins (np.ndarray): lower cutoff in the power law
                of whiff durations, for each odor
            whiff_tmaxs (np.ndarray): upper cutoff in the power law
                of whiff durations, for each odor
            blank_tmins (np.ndarray): same as whiff_tmins but for blanks
            blank_tmaxs (np.ndarray): same as whiff_tmaxs but for blanks
            c0s (np.ndarray): c0 concentration scale for each odor
            alphas (np.ndarray): alpha*c0 is the lower cutoff of p_c
            vecs (np.ndarray): 3d array where axis 0 has length 2, 
                the first sub-array giving the kappa vectors of each odor, 
                and the second, giving eta vectors. 
        noises (np.ndarray): fresh U(0, 1) samples, shaped [n_odors, 2],
            in case we need to pull a new t and/or c.
            TODO: most noises are wasted; for now memory isn't an issue
            but this is a place where the code can be optimized a lot.
        dt (float): time step duration, in simulation units
    """
    # Update one odor's t and c at a time, if necessary
    # TODO: reorder to have back_components first, then all params of each odor
    # Would make the update faster, because we update one odor at a time. 
    tc_bk_new = np.zeros(tc_bk.shape)
    for i in range(tc_bk.shape[0]):
        tc_bk_new[i] = update_tc_odor(tc_bk[i], dt, noises[i],
                                *[p[i] for p in params_bk[:-1]])

    # Compute backgound vector (even if it didn't change)
    vecs_nu = params_bk[-1]  # kappavecs, etavecs
    new_bk_vec = combine_odors_ornmodel(tc_bk_new[:, 1], vecs_nu[0], vecs_nu[1], n_cng=4, fmax=1.0)
    return new_bk_vec, tc_bk_new

## Initialization

### Aesthetic parameters

In [ ]:
#plt.style.use(['dark_background'])
plt.rcParams["figure.figsize"] = (4.5, 3.0)

In [ ]:
models = ["ibcm", "biopca", "avgsub", "ideal", "orthogonal", "none"]
model_nice_names = {
    "ibcm": "IBCM",
    "biopca": "BioPCA",
    "avgsub": "Average",
    "ideal": "Ideal",
    "orthogonal": "Orthogonal",
    "none": "None"
}
model_colors = {
    "ibcm": "xkcd:turquoise",
    "biopca": "xkcd:orangey brown",
    "avgsub": "xkcd:navy blue",
    "ideal": "xkcd:powder blue",
    "orthogonal": "xkcd:pale rose",
    "none": "grey"
}

### Global model parameters

In [ ]:
# Initialize common simulation parameters
n_dimensions = 25  # Half the real number for faster simulations
n_components = 3  # Number of background odors

inhib_rates = [0.0001, 0.00002]  # alpha, beta  [0.00025, 0.00005]

# Simulation duration
duration = 360000.0
deltat = 1.0
n_chunks = 10
skp = 10 * int(1.0 / deltat)

# Common model options
activ_function = "identity"  #"ReLU"

# Choose randomly generated background vectors
rgen_meta = np.random.default_rng(seed=0x270369e90599ffa80a743d99ac942f28)

# Background process
update_fct = update_powerlaw_times_concs_ornmodel
# Seed for background simulation, to make sure all models are the same
simul_seed = seed_from_gen(rgen_meta)

# Turbulent background parameters: same rates and constants for all odors
back_params = [
    np.asarray([1.0] * n_components),        # whiff_tmins
    np.asarray([500.] * n_components),       # whiff_tmaxs
    np.asarray([1.0] * n_components),        # blank_tmins
    np.asarray([800.0] * n_components),      # blank_tmaxs
    np.asarray([0.6] * n_components),        # c0s
    np.asarray([0.5] * n_components),        # alphas
]

# Background odors: parameters (kappa^-1, eta) and x_gamma vectors
rho_back = 0.5
std_kappa = 4.0
# Indexed [kappa_or_eta, n_odors, n_dimensions]
back_components = [np.asarray(generate_odorant_kappaeta(rho_back, n_dimensions, std_kappa, rgen_meta)) 
                   for _ in range(n_components)]
back_components = np.stack(back_components, axis=1)
back_params.append(back_components)
# We define x_gamma to be the ORN activation in response to odor gamma at large concentration 10
x_gamma_vecs = np.asarray([combine_odors_ornmodel(np.ones(1)*10.0, back_components[0, i], back_components[1, i], 
                                       n_cng=4, fmax=1.0) for i in range(n_components)])
print(x_gamma_vecs)

# Initial values of background process variables (t, c for each variable)
init_concs = sample_ss_conc_powerlaw(*back_params[:-1], size=1, rgen=rgen_meta)
init_times = powerlaw_cutoff_inverse_transform(
                rgen_meta.random(size=n_components), *back_params[2:4])
tc_init = np.stack([init_times, init_concs.squeeze()], axis=1)

# Initial background vector: combine odors with the tc_init concentrations
init_bkvec = combine_odors_ornmodel(tc_init[:, 1], back_components[0], back_components[1], n_cng=4, fmax=1.0)
# nus are first in the list of initial background params
init_back_list = [tc_init, init_bkvec]

## IBCM habituation
### IBCM simulation

The good question is: what are IBCM neurons going to learn? The input space is $\vec{x}$, but it's not a linear combination of background odors, which are rather defined by $\vec{\eta}$ and $\vec{\kappa}$. So it's unclear even what exactly will be the input process distribution, what components will be learnt, etc. 

In [ ]:
# IBCM model parameters
n_i_ibcm = 24  # Number of inhibitory neurons for IBCM case

# Model rates
learnrate_ibcm = 0.0003  #5e-5
tau_avg_ibcm = 1200  # 2000
coupling_eta_ibcm = 0.5/n_i_ibcm
ssat_ibcm = 50.0
k_c2bar_avg = 0.1
decay_relative_ibcm = 0.005
lambd_ibcm = 1.0
ibcm_rates = [
    learnrate_ibcm, 
    tau_avg_ibcm, 
    coupling_eta_ibcm, 
    lambd_ibcm,
    ssat_ibcm, 
    k_c2bar_avg,
    decay_relative_ibcm 
]
ibcm_options = {
    "activ_fct": activ_function, 
    "saturation": "tanh", 
    "variant": "law", 
    "decay": True
}

# Initial synaptic weights: small positive noise
init_synapses_ibcm = 0.3*rgen_meta.standard_normal(size=[n_i_ibcm, n_dimensions])*lambd_ibcm

In [ ]:
# Run the IBCM simulations
# Perform successive shorter runs/restarts for memory efficiency
tser_ibcm = []
nuser_ibcm = []
bkvecser_ibcm = []
mser_ibcm = []
cbarser_ibcm = []
wser_ibcm = []
sser_ibcm = []
thetaser_ibcm = []
if n_chunks > 1:
    seed_spawns = np.random.SeedSequence(simul_seed).spawn(10)
else:
    seed_spawns = [simul_seed]
for i in range(n_chunks):
    tstart = perf_counter()
    if i == 0:
        init_vari = init_synapses_ibcm
        init_back = init_back_list
    else:
        init_vari = [mser_ibcm[i-1][-1], thetaser_ibcm[i-1][-1], wser_ibcm[i-1][-1]]
        init_back = [nuser_ibcm[i-1][-1], bkvecser_ibcm[i-1][-1]]
    sim_results = integrate_inhib_ibcm_network_options(
                init_vari, update_fct, init_back, 
                ibcm_rates, inhib_rates, back_params, duration/n_chunks, 
                deltat, seed=seed_spawns[i], noisetype="uniform",  
                skp=skp, **ibcm_options
    )
    tser_ibcm.append(sim_results[0] + i/n_chunks*duration)
    nuser_ibcm.append(sim_results[1])
    bkvecser_ibcm.append(sim_results[2])
    mser_ibcm.append(sim_results[3]) 
    cbarser_ibcm.append(sim_results[4]) 
    thetaser_ibcm.append(sim_results[5])
    wser_ibcm.append(sim_results[6])
    sser_ibcm.append(sim_results[7])
    tend = perf_counter()
    print("Finished chunk", i, "in {:.2f} s".format(tend - tstart))

# Concatenate
tser_ibcm = np.concatenate(tser_ibcm, axis=0)
nuser_ibcm = np.concatenate(nuser_ibcm)
bkvecser_ibcm = np.concatenate(bkvecser_ibcm)
mser_ibcm = np.concatenate(mser_ibcm)
cbarser_ibcm = np.concatenate(cbarser_ibcm)
thetaser_ibcm = np.concatenate(thetaser_ibcm)
wser_ibcm = np.concatenate(wser_ibcm)
sser_ibcm = np.concatenate(sser_ibcm)

### Background process plot

In [ ]:
if n_dimensions <= 2:
    fig, ax = plt.subplots()
    # Use a different color for 3 cases: either odor absent, both odors
    where_12 = (nuser_ibcm[:, :, 1] > 0).astype(bool)
    where_both = np.logical_and(where_12[:, 0], where_12[:, 1])
    where_1 = np.logical_and(where_12[:, 0], ~where_12[:, 1])
    where_2 = np.logical_and(~where_12[:, 0], where_12[:, 1])
    ax.scatter(bkvecser_ibcm[where_both, 0], bkvecser_ibcm[where_both, 1], label="Both odors")
    ax.scatter(bkvecser_ibcm[where_1, 0], bkvecser_ibcm[where_1, 1], label="Odor 0 only")
    ax.scatter(bkvecser_ibcm[where_2, 0], bkvecser_ibcm[where_2, 1], label="Odor 1 only")
    vecs = np.zeros(x_gamma_vecs.shape)
    scale = 0.5
    orig = np.zeros([3, n_components])
    for i in range(n_components):
        vecs[i] = x_gamma_vecs[i] / np.sqrt(np.sum(x_gamma_vecs[i]**2)) * scale
        ax.annotate("", xytext=(0, 0), xy=x_gamma_vecs[i], 
                    arrowprops=dict(width=2.0, color="k"))
    figname = "figures/powerlaw/background_two_odors_2d_ornmodel.pdf"
    zlbl = None
elif n_components == 3:
    dims = (4, 5, 6)
    fig = plt.figure()
    ax = fig.add_subplot(projection="3d")
    fig.set_size_inches(6.0, 3.0)
    where_123 = (nuser_ibcm[:, :, 1] > 0).astype(bool)
    locations = {
        "All odors": np.all(where_123, axis=1),  #all
        "Odors 0&1": (where_123[:, 0] & where_123[:, 1] & ~where_123[:, 2]),  # 12
        "Odors 0&2": (where_123[:, 0] & ~where_123[:, 1] & where_123[:, 2]),  # 13
        "Odors 1&2": (~where_123[:, 0] & where_123[:, 1] & where_123[:, 2]),  # 23
        "Odor 0 ": (where_123[:, 0] & ~where_123[:, 1] & ~where_123[:, 2]),  #1
        "Odor 1": (~where_123[:, 0] & where_123[:, 1] & ~where_123[:, 2]), 
        "Odor 2": (~where_123[:, 0] & ~where_123[:, 1] & where_123[:, 2])
    }
    for lbl, slc in locations.items():
        ax.scatter(bkvecser_ibcm[slc, dims[0]], bkvecser_ibcm[slc, dims[1]], 
                   bkvecser_ibcm[slc, dims[2]], label=lbl)
    vecs = np.zeros(x_gamma_vecs.shape)
    scale = 0.5
    orig = np.zeros([3, n_components])
    for i in range(n_components):
        vecs[i] = x_gamma_vecs[i] / np.sqrt(np.sum(x_gamma_vecs[i]**2)) * scale
    ax.quiver(*orig, *(x_gamma_vecs[:, dims].T), color="k", lw=2.0)
    ax.scatter(0, 0, 0, color="k", s=100)
    #ax.view_init(azim=30, elev=30)
    figname = "figures/powerlaw/background_three_odors_3d_ornmodel.pdf"
    zlbl = ax.set_zlabel("ORN {}".format(dims[2]))
elif n_components == 2:
    dims = (0, 1, 4)
    fig = plt.figure()
    ax = fig.add_subplot(projection="3d")
    fig.set_size_inches(6.0, 3.0)
    where_12 = (nuser_ibcm[:, :, 1] > 0).astype(bool)
    locations = {
        "Both odors": np.all(where_12, axis=1),  # all
        "Odor 0 ": (where_12[:, 0] & ~where_12[:, 1]),  # 1
        "Odor 1": (~where_12[:, 0] & where_12[:, 1])  # 2
    }
    for lbl, slc in locations.items():
        ax.scatter(bkvecser_ibcm[slc, dims[0]], bkvecser_ibcm[slc, dims[1]], 
                   bkvecser_ibcm[slc, dims[2]], label=lbl)
    vecs = np.zeros(x_gamma_vecs.shape)
    scale = 0.5
    orig = np.zeros([3, n_components])
    for i in range(n_components):
        vecs[i] = x_gamma_vecs[i] / np.sqrt(np.sum(x_gamma_vecs[i]**2)) * scale
    ax.quiver(*orig, *(x_gamma_vecs[:, dims].T), color="k", lw=2.0)
    ax.scatter(0, 0, 0, color="k", s=100)
    #ax.view_init(azim=30, elev=30)
    figname = "figures/powerlaw/background_two_odors_3d_ornmodel.pdf"
    ax.set(xlabel="ORN {}".format(dims[0]), ylabel="ORN {}".format(dims[1]))
    zlbl = ax.set_zlabel("ORN {}".format(dims[2]))
leg = ax.legend(loc="upper right", bbox_to_anchor=(0.0, 1.0))
fig.tight_layout()
#fig.savefig(figname, transparent=True, bbox_inches="tight", bbox_extra_artists=(zlbl, leg))
plt.show()
plt.close()

### Plotting the time course of the different neurons

In [ ]:
# Calculate cgammas_bar and mbars
transient = int(5/6*duration / deltat) // skp
# Dot products \bar{c}_{\gamma} = \bar{\vec{m}} \cdot \vec{x}_{\gamma}
mbarser, c_gammas, cbars_gamma = compute_mbars_cgammas_cbargammas(
                                    mser_ibcm, coupling_eta_ibcm, x_gamma_vecs)
sums_cbars_gamma = np.sum(cbars_gamma, axis=2)
sums_cbars_gamma2 = np.sum(cbars_gamma*cbars_gamma, axis=2)

# Analytical prediction, exact: need moments of nu. Easiest to compute numerically. 
conc_ser = nuser_ibcm[:, :, 1]
# Odors are all iid so we can average over all odors
mean_conc = np.mean(conc_ser)
sigma2_conc = np.var(conc_ser)
thirdmom_conc = np.mean((conc_ser - mean_conc)**3)
moments_conc = [mean_conc, sigma2_conc, thirdmom_conc]

# Analytical prediction
res = fixedpoint_thirdmoment_exact(moments_conc, 1, n_components-1)
c_specif, c_nonspecif = res[:2]
cs_cn = res[:2]

# Count how many dot products are at each possible value. Use cbar = 1.0 as a split. 
split_val = 2.0
cbars_gamma_mean = np.mean(cbars_gamma[transient:], axis=0)
cgammas_bar_counts = {"above": int(np.sum(cbars_gamma_mean.flatten() > split_val)), 
                      "below": int(np.sum(cbars_gamma_mean.flatten() <= split_val))}
print(cgammas_bar_counts)

specif_gammas = np.argmax(np.mean(cbars_gamma[transient:], axis=0), axis=1)
print(specif_gammas)

# Analytical W
analytical_w = ibcm_fixedpoint_w_thirdmoment(inhib_rates, moments_conc, x_gamma_vecs, cs_cn, specif_gammas)

### IBCM habituation analysis

In [ ]:
fig, ax = plt.subplots()
#ax.plot(tser_ibcm[:300], nuser_ibcm[:300, :, 1])
neurons_cmap = sns.color_palette("Greys", n_colors=n_i_ibcm)
for i in range(n_i_ibcm):
    ax.plot(tser_ibcm/1000, thetaser_ibcm[:, i], lw=0.5, color=neurons_cmap[i])
ax.set(xlabel="Time (x1000 steps)", ylabel=r"$\bar{\Theta} = \bar{c}^2$ moving average")
plt.show()
plt.close()

In [ ]:
fig , ax, _ = plot_cbars_gamma_series(tser_ibcm, cbars_gamma, 
                        skp=10, transient=320000 // skp)
# Compare to exact analytical fixed point solution
#ax.set_xlim([350, 360])
ax.axhline(c_specif, ls="--", color="grey", 
           label=r"Analytical $\bar{c}_{\gamma=\mathrm{specific}}$")
ax.axhline(c_nonspecif, ls="--", color="grey", 
           label=r"Analytical $\bar{c}_{\gamma=\mathrm{non}}$")
fig.tight_layout()
leg = ax.legend(loc="upper left", bbox_to_anchor=(1., 1.))

#fig.savefig("figures/powerlaw/cbargammas_series_turbulent_background_example.pdf", 
#            transparent=True, bbox_inches="tight", bbox_extra_artists=(leg,))
plt.show()
plt.close()

In [ ]:
# Correlation between nu's and c's, see if some neurons are specific to odors
# Each neuron turns out to correlate its response to  one concentration
# that means it is specific to that odor. 
cbarser_norm_centered = cbarser_ibcm - np.mean(cbarser_ibcm[transient:], axis=0)
conc_ser_centered = (nuser_ibcm[:, :, 1] 
                     - np.mean(nuser_ibcm[transient:, :, 1], axis=0))
correl_c_nu = np.mean(cbarser_norm_centered[transient:, :, None] 
                      * conc_ser_centered[transient:, None, :], axis=0)

fig, ax = plt.subplots()
img = ax.imshow(correl_c_nu.T)
ax.set(ylabel=r"Component $\gamma$", xlabel=r"Neuron $i$")
fig.colorbar(img, label=r"$\langle (\bar{c}^i - \langle \bar{c}^i \rangle)"
             r"(\nu_{\gamma} - \langle \nu_{\gamma} \rangle) \rangle$", 
            location="top")
fig.tight_layout()
#fig.savefig("figures/powerlaw/specificities_turbulent_background_example.pdf", 
#           transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Check if each component has at least one neuron
for comp in range(n_components):
    print("Number of neurons specific to component {}: {}".format(
            comp, np.sum(np.mean(cbars_gamma[-2000:, :, comp], axis=0) > split_val)))

In [ ]:
fig, axes, _ = plot_background_neurons_inhibition(tser_ibcm, bkvecser_ibcm, sser_ibcm, skp=1)
plt.show()
plt.close()

In [ ]:
fig, ax, bknorm_ser, snorm_ser = plot_background_norm_inhibition(
                                tser_ibcm, bkvecser_ibcm, sser_ibcm, skp=1)

# Compute noise reduction factor, annotate
transient = 100000 // skp
norm_stats = compute_back_reduction_stats(bknorm_ser, snorm_ser, trans=transient)

print("Mean activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['avg_reduction'] * 100))
print("Standard deviation of activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['std_reduction'] * 100))
ax.annotate("St. dev. reduced to {:.1f} %".format(norm_stats['std_reduction'] * 100), 
           xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
fig.tight_layout()
#fig.savefig("figures/powerlaw/pn_activity_norm_turbulent_background_example.pdf", 
#            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
fig, axes = plot_w_matrix(tser_ibcm, wser_ibcm, skp=100)
fig.tight_layout()
#fig.savefig("figures/powerlaw/w_series_turbulent_background_example.pdf", 
#            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

### Stability of average fixed points
Check the eigenvalues of the jacobian for one neuron, for every possible specificity. There are $2^{n_B}$ possibilities: choosing specific or not for each odor

That calculation does not really work here, since the background process is not at all the linear superposition used analytically. 

In [ ]:
all_max_eigenvalues = ibcm_all_largest_eigenvalues(
    moments_conc, ibcm_rates, x_gamma_vecs, m3=1.0, cut=1e-16, options=ibcm_options
)

In [ ]:
fig, ax = plt.subplots()
ibcm_specif_keys = list(all_max_eigenvalues.keys())
ibcm_eig_values = np.asarray([all_max_eigenvalues[a] for a in ibcm_specif_keys])
reals, imags = np.real(ibcm_eig_values), np.imag(ibcm_eig_values)
ibcm_eig_values_specif1 = np.asarray([len(s) == 1 for s in ibcm_specif_keys], dtype=bool)
highlights = ibcm_eig_values_specif1
ax.axvline(0.0, ls="--", color="k", lw=1.0)
ax.axhline(0.0, ls="--", color="k", lw=1.0)
scaleup = 1e3
ax.plot(reals[highlights]*scaleup, imags[highlights]*scaleup, marker="*", mfc="b", mec="b", 
        ls="none", label="One odor", ms=8)
ax.plot(reals[~highlights]*scaleup, imags[~highlights]*scaleup, marker="o", mfc="k", mec="k", 
       ls="none", label="0 or 2+ odors", ms=6)
for side in ("top", "right"):
    ax.spines[side].set_visible(False)
ax.legend(title="Specificity")
ax.set(xlabel=r"$\mathrm{Re}(\lambda_{\mathrm{max}})$    ($\times 10^{-3}$)", 
      ylabel=r"$\mathrm{Im}(\lambda_{\mathrm{max}})$     ($\times 10^{-3}$)")
fig.tight_layout()
plt.show()
plt.close()

## BioPCA simulation

### BioPCA habituation simulation

In [ ]:
# BioPCA model parameters
n_i_pca = n_components*2  # Number of inhibitory neurons for BioPCA case

# Model rates
learnrate_pca = 1e-4  # Learning rate of M
# Choose Lambda diagonal matrix as advised in Minden et al., 2018
# but scale it up to counteract W regularization
lambda_range_pca = 0.5
lambda_max_pca = 8.0
# Learning rate of L, relative to learnrate. Adjusted to Lambda in the integration function
rel_lrate_pca = 2.0  #  / lambda_max_pca**2 
lambda_mat_diag = build_lambda_matrix(lambda_max_pca, lambda_range_pca, n_i_pca)

xavg_rate_pca = learnrate_pca
pca_options = {
    "activ_fct": activ_function, 
    "remove_lambda": False, 
    "remove_mean": True
}
biopca_rates = [learnrate_pca, rel_lrate_pca, lambda_max_pca, lambda_range_pca, xavg_rate_pca]


# Initial synaptic weights: small positive noise
# We selected a seed (out of 40+ tested) giving initial conditions leading to correct PCA
# The model has trouble converging on this background, we're giving as many chances as possible here. 
rgen_pca = np.random.default_rng(seed=0x8b6664612cfeda4a121436fcfbbca449)
init_synapses_pca = rgen_pca.standard_normal(size=[n_i_pca, n_dimensions]) / np.sqrt(n_i_pca)
init_mmat_pca = rgen_pca.standard_normal(size=[n_i_pca, n_dimensions]) / np.sqrt(n_dimensions)
init_lmat_pca = np.eye(n_i_pca, n_i_pca)  # Supposed to be near-identity, start as identity
ml_inits_pca = [init_mmat_pca, init_lmat_pca]

In [ ]:
# Run simulation
sim_results = integrate_inhib_ifpsp_network_skip(
                ml_inits_pca, update_fct, init_back_list, biopca_rates, 
                inhib_rates, back_params, duration, deltat, 
                seed=simul_seed, noisetype="uniform", skp=skp, **pca_options)
(tser_pca, 
 nuser_pca, 
 bkvecser_pca, 
 mser_pca, 
 lser_pca, 
 xser_pca, 
 cbarser_pca, 
 wser_pca, 
 sser_pca) = sim_results

### BioPCA simulation analysis

In [ ]:
res = analyze_pca_learning(bkvecser_pca, mser_pca, lser_pca, 
                           lambda_mat_diag, demean=pca_options["remove_mean"])
true_pca, learnt_pca, fser, off_diag_l_avg_abs, align_error_ser = res

In [ ]:
from utils.statistics import principal_component_analysis
from modelfcts.checktools import compute_pca_meankept

In [ ]:
fig, axes = plot_pca_results(tser_pca/1000, true_pca, learnt_pca, align_error_ser, off_diag_l_avg_abs)
axes[-1].set_xlabel("Time (x1000 steps)")
fig.set_size_inches(fig.get_size_inches()[0], 3*2.5)
plt.show()
plt.close()

In [ ]:
fig, ax, bknorm_ser, snorm_ser = plot_background_norm_inhibition(
                                tser_pca, bkvecser_pca, sser_pca, skp=10)

# Compute noise reduction factor, annotate
transient = 100000 // skp
norm_stats = compute_back_reduction_stats(bknorm_ser, snorm_ser, trans=transient)

print("Mean activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['avg_reduction'] * 100))
print("Standard deviation of activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['std_reduction'] * 100))
ax.annotate("St. dev. reduced to {:.1f} %".format(norm_stats['std_reduction'] * 100), 
           xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
fig.tight_layout()
plt.show()
plt.close()

### Response to a new odor
This part of the code only runs if the simulation above had ``n_dimensions > n_components``. 

The goal is to see whether a new odor, not linearly dependent of the ones in the background, also gets repressed close to zero, or produces an inhibited output noticeably different from the inhibited background, and still similar to the new odor vector, at least its component perpendicular to the background subspace. 

Need to test for many samples from the background odor distribution. Keep the new odor at a constant concentration, typical of the concentration at which we actually want the system to pick up the new odor. 

I realize that it's fine if the disentanglement of odors isn't perfect at the PN layer: besides the question of habituation, the sparse tag network proposed by Dasgupta does not address too well how multiple odors are disentangled from a complicated mixture. 

In [ ]:
def respond_new_odors(odors, typical_m, typical_w, coupling):
    """ 
    Args:
        odors (np.ndarray): indexed [..., dimension] 
            so can take dot product properly with m and store many 
            odors along arbitrary other axes.
        typical_m (np.ndarray): indexed [n_neurons, dimension]
        typical_w (np.ndarray): indexed [n_neurons, dimension]
    """
    # Compute activation of neurons to each new odor (new+background)
    # Given the IBCM and inhibitory neurons' current state 
    # (either latest or some average state of the neurons)
    c = odors.dot(typical_m.T)
    cbar = c - coupling*(np.sum(c, axis=-1, keepdims=True) - c)  # -c to cancel the subtraction of c[i] itself
    print(cbar)
    # Inhibit with the mean cbar*wser, to see how on average the new odor will show
    n_neu = typical_m.shape[0]
    new_outputs = odors - np.sum(np.expand_dims(cbar, cbar.ndim)*typical_w[np.newaxis, :], axis=-2)/n_neu
    return new_outputs

In [ ]:
def l2_norm(vecs):
    """ Computes l2 norm of vectors stored along the last axis of vecs. 
    Args:
        vecs can be either a single vector (1d) or an  arbitrary array of vectors, 
            where the last dimension indexes elements of vectors. 
    
    Returns: if vecs is of shape (K x L x ... M x N), 
        returns an array of distances of shape (K x L x ... x M) 
    """
    return np.sqrt(np.sum(vecs**2, axis=-1))

def l1_norm(vecs):
    """ |x| = \sum_i |x_i|"""
    return np.sum(np.abs(vecs), axis=-1)

def linf_norm(vecs):
    """ |x| = max_i(|x_i|) """
    return np.max(np.abs(vecs), axis=-1)

def cosine_dist(x, y):
    """ d(x, y) = 1 - (x \cdot y)/(|x| |y|)"""
    xnorm, ynorm = l2_norm(x), l2_norm(y)
    return 1.0 - x.dot(np.moveaxis(y, -1, 0)) / xnorm / ynorm

def distance_panel_target(mixes, target):
    """ Compute a panel of distances between the pure (target) new odor and mixtures 
    (which can be without inhibition, with average inhibition, IBCM inhibition, etc.). 
    
    Four distances included, in order: l2, l1, linf, cosine_dist
    
    Args:
        mixes (np.ndarray): mixtures of odors to compute distance from target, 
            the last axis should have the size of target, 
            while other axes are arbitrary.  
        target (np.1darray): target odor vector, same length as
            last axis of mixes. 
    Returns:
        dist_panel (np.ndarray): shape of pure, except the last axis, 
            which has length 4 (for the number of distances computed). 
    """
    # Make axis 0 the axis indexing distance metrics, to begin with
    # And move it to the last axis before returning
    dist_array = np.zeros([4] + list(mixes.shape[:-1]))
    # No need to add axes to target vector; if it is 1d, it is broadcasted
    # along the last axis of mixes, which indexes elements of each vector. 
    dist_array[0] = l2_norm(target - mixes)
    dist_array[1] = l1_norm(target - mixes)
    dist_array[2] = linf_norm(target - mixes)
    dist_array[3] = cosine_dist(target, mixes)
    
    return np.moveaxis(dist_array, 0, -1)

In [ ]:
# Statistics of improvement of recognition
# New odor
new_odor = np.roll(back_components_sym[0], shift=-1)  # Should be a new vector

# mbar dot product with the new odor
cdot_new = mser_sat.dot(new_odor)
cbardot_new = cdot_new*(1 + coupling_eta) - coupling_eta*np.sum(cdot_new, axis=1, keepdims=True)

# Background samples, then add new odor
typical_conc = np.mean(back_params_sym[-3]) * np.mean(back_params_sym[-2])
mix_samples = sample_background_powerlaw(back_components_sym, *back_params_sym[:-1], size=1000, rgen=rgen_meta)
mix_samples = 0.8 * mix_samples + 0.2 * new_odor.reshape(1, -1) * typical_conc
mix_samples = np.tanh(mix_samples / 0.5)

# Compare to inhibition of the average background
#avg_back = averages_nu.dot(back_components_sym)
#a_over_ab = inhib_rates[0] / sum(inhib_rates)
#inhib_avg_samples = mix_samples - a_over_ab * avg_back.reshape(1, -1)

# Average m and w with which we will inhibit
msat_mean = np.mean(mser_sat[transient:], axis=0)
wsat_mean = np.mean(wser_sat[transient:], axis=0)

# Inhibition of each generated sample and statistics on performance
inhib_ibcm_samples = respond_new_odors(mix_samples, msat_mean, wsat_mean, coupling_eta)
print(np.tanh(0.5*typical_conc*new_odor / 0.5))
print(inhib_ibcm_samples)
print(mix_samples)

dist_pure_inhib_none = distance_panel_target(mix_samples, np.tanh(0.2* typical_conc*new_odor / 0.5))
#dist_pure_inhib_avg = distance_panel_target(inhib_avg_samples, new_odor)
dist_pure_inhib_ibcm = distance_panel_target(inhib_ibcm_samples, np.tanh(0.2*typical_conc*new_odor / 0.5))

median_distances_none = np.median(dist_pure_inhib_none, axis=0)
#median_distances_avg = np.median(dist_pure_inhib_avg, axis=0)
median_distances_ibcm = np.median(dist_pure_inhib_ibcm, axis=0)

In [ ]:
# Histogram of distance to pure odor, for each distance
# Overlay histogram for mix without and with inhibition
fig, axes = plt.subplots(2, 2)
axes = axes.flatten()
clr_none = "xkcd:navy blue"
clr_ibcm = "xkcd:turquoise"
clr_avg = "xkcd:orangey brown"
dist_names = [r"$L^2$ distance", r"$L^1$ distance", r"$L^{\infty}$ distance", "Cosine distance"]
for i, ax in enumerate(axes):
    ax.hist(dist_pure_inhib_none[:, i], label="No inhibition", facecolor=clr_none, alpha=0.6, 
        edgecolor=clr_none, density=True)
    ax.axvline(median_distances_none[i], color=clr_none, ls="--", lw=1.0)
    #ax.hist(dist_pure_inhib_avg[:, i], label="Average inhibition", facecolor=clr_avg, alpha=0.6, 
    #    edgecolor=clr_avg, density=True)
    #ax.axvline(median_distances_avg[i], color=clr_avg, ls="--", lw=1.0)
    ax.hist(dist_pure_inhib_ibcm[:, i], label="IBCM inhibition", facecolor=clr_ibcm, alpha=0.6, 
        edgecolor=clr_ibcm, density=True) 
    ax.axvline(median_distances_ibcm[i], color=clr_ibcm, ls="--", lw=1.0)
    ax.set(xlabel="Distance to new odor", ylabel="Probability density", title=dist_names[i])
axes[0].legend()
fig.tight_layout()
plt.show()
plt.close()

In [ ]:
#np.mean(cbardot_new[transient:], axis=0)
mbarser_sat = mser_sat*(1.0 + coupling_eta) - coupling_eta*np.sum(mser_sat, axis=1, keepdims=True)
mbarser_sat_mean = np.mean(mbarser_sat[transient:], axis=0)
#print(mbarser_sat_mean)
full_basis = np.vstack([back_components_sym, new_odor])
#print(full_basis)
mbarser_sat_decomposed = np.asarray([decompose_nonorthogonal_basis(mbarser_sat_mean[i], full_basis) 
                                     for i in range(n_neurons)])
print(mbarser_sat_decomposed)

In [ ]:
mser_sat_decomposed = np.asarray([decompose_nonorthogonal_basis(np.mean(mser_sat, axis=0)[i], full_basis) 
                                     for i in range(n_neurons)])
print(mser_sat_decomposed)
# Because of the tanh transform, the component orthogonal to the background basis
# slowly changes nevertheless. 
# Idea: add a small decay term -mu*m to the IBCM equations to force components of m orthogonal to the
# background to disappear before a new odor appears. 

minit_bar = init_synapses * (1.0 + coupling_eta) - coupling_eta * np.sum(init_synapses, axis=0, keepdims=True)
minit_bar_decomposed = np.asarray([decompose_nonorthogonal_basis(minit_bar[i], full_basis) 
                                     for i in range(n_neurons)])
minit_decomposed = np.asarray([decompose_nonorthogonal_basis(init_synapses[i], full_basis) 
                                     for i in range(n_neurons)])
print(minit_decomposed)

In [ ]:
# Average dot product of mbars with the new odor
np.mean(mbarser_sat_mean.dot(new_odor))
#np.mean(mser_sat, axis=0).dot(new_odor)
# That's not zero at all, even on average, which causes problems when the time comes
# to inhibit the background plus odor mix. Not sure why. Need to fix that. 

In [ ]:
# Decompose the w_i, check whether they have a component not in the subspace of the x_gamma of the background
wmean_sat = np.mean(wser_sat[transient:], axis=0)
w_sat_decomp = np.asarray([decompose_nonorthogonal_basis(wmean_sat[i], full_basis) for i in range(n_neurons)])
wmean_sat_norm = wmean_sat / np.sqrt(np.sum(wmean_sat**2, axis=1, keepdims=True))
w_sat_norm_decomp = np.asarray([decompose_nonorthogonal_basis(wmean_sat_norm[i], full_basis)
                                for i in range(n_neurons)])
#print(wmean_sat)
print(w_sat_decomp[:4])
#print(wmean_sat_norm)
#print(w_sat_norm_decomp)

In [ ]:
sum_cgammas_w = np.dot(cgammas_bar_mean_sat[[0, 1, 2]].T, wmean_sat[[0, 1, 2]]) / 3
#[decompose_nonorthogonal_basis(sum_cgammas_w[i], full_basis) for i in range(n_components)]
back_components_sym - sum_cgammas_w

In [ ]:
np.sum(cgammas_bar_mean_sat == cgammas_bar_mean_sat.max(axis=1, keepdims=True), axis=0)

In [ ]:
wi_basis = np.vstack([wmean_sat[[0, 1, 2]], new_odor])

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
vecs2 = wmean_sat[[0, 1, 2]]
vecs2 = vecs2 / np.sqrt(np.sum(vecs2**2, axis=1, keepdims=True))
vecs = vecs / np.sqrt(np.sum(vecs**2, axis=1, keepdims=True))
ax.quiver(*(orig[0:3]), *(vecs2[:, 0:3].T), color="b", lw=2.0)
ax.quiver(*(orig[0:3]), *(vecs[:, 0:3].T), color="k", lw=2.0)
ax.set_xlim(0.0, max(vecs2.max(), vecs.max()))
ax.set_ylim(0.0, max(vecs2.max(), vecs.max()))
ax.set_zlim(0.0, max(vecs2.max(), vecs.max()))
ax.view_init(azim=60, elev=30)
plt.show()
plt.close()

In [ ]:
raise NotImplementedError("Stop here for now")

#### Metric to measure the quality of the inhibition
Distance to new odor alone? Compare to un-inhibited mixture?

Ultimately, will compute sparse binary neural tag and compare with Jaccard metric, but for now, I want to avoid this complication, which requires using many more dimensions than 4. I might keep this for a separate notebook (or even C code if it seems to work well). 

### 1.b) Random odor vectors

In [ ]:
# Realistic model of olfactory receptor activation patterns:
# each component is i.i.d. exponential
def generate_odorant(n_rec, rgen, lambda_in=0.1):
    """ Generate vectors eta and kappa^-1 for an odorant, with antagonism parameter rho. 
    
    Args:
        n_rec (int): number of receptor types, length of vectors
        rgen (np.random.Generator): random generate (numpy >= 1.17)
        lambda_in (float): lambda parameter of the exp distribution
            Equals the inverse of the average of each vector component
    Returns:
        kappa1_vec (np.ndarray): 1d vector of receptor activities
    """
    return rgen.exponential(scale=1.0/lambda_in, size=n_rec)

## 2. Complete model: sparse Kenyon cell tags for odors
We need to make new simulations with many more dimensions (ORN types). 

Consequently, to avoid running into memory issues, we only save a subset of time steps in the simulation: this is fine because we are only interested in the slowly-evolving $\vec{w}$ and $\vec{m}$, while we don't care too much for $\vec{x}$'s fast fluctuations. We just want the final average $\vec{w}$ to apply as inhibition to randomly sampled background odors, which we don't even take from simulations but just generate from the steady-state distribution. 

### Run a new simulation with 25 dimensions

In [ ]:
### General simulation parameters
n_dimensions_tag = 25  # Half the real number for faster simulations
n_neurons = 64

# Simulation rates and coupling stay the same (try at least)
duration = 160000.0
deltat = 1.0
tau_nu = 2.0  # Correlation time scale of the background nu_gammas (same for all)
learnrate = 0.005
tau_avg = 25
inhib_rates = [0.0025, 0.0005]  # alpha, beta
# Background components need to be redefined. Extra dimensions are somewhat superfluous
coupling_eta = 0.5/n_neurons

# Choose symmetric, normalized background odor components
#back_components_tag = np.ones([n_components, n_dimensions_tag]) * 0.2
#for i in range(n_components):
#    back_components_tag[i, i] = 0.8
#    back_components_tag[i] /= np.sqrt(np.sum(back_components_tag[i]**2))

# Choose randomly generated background vectors
rgen_meta_tag = np.random.default_rng(seed=38981211111)
back_components_tag = np.zeros([n_components, n_dimensions_tag])
for i in range(n_components):
    back_components_tag[i] = generate_odorant(n_dimensions_tag, rgen_meta_tag, lambda_in=0.1)
print(back_components_tag)
back_components_tag = back_components_tag / l2_norm(back_components_tag).reshape(-1, 1)
    
# Initial synaptic weights: small positive noise
init_synapses_tag = 0.1*rgen_meta.random(size=[n_neurons, n_dimensions_tag])

# Initial nu values stay the same
init_bkvec_tag = averages_nu.dot(back_components_tag)
# nus are first in the list of initial background params
init_back_list_tag = [init_nu, init_bkvec_tag]

# Update matrices for nu process stay the same
back_params_tag = [update_mat_A, update_mat_B, back_components_tag, averages_nu, epsilon_nu]

In [ ]:
# Run a heavy simulation
skp_tag = 20
sim_results = integrate_inhib_ibcm_network_skip(init_synapses_tag, update_thirdmoment_kinputs, init_back_list_tag, 
                    inhib_rates, back_params_tag, duration, deltat, learnrate=learnrate, seed=73001317, 
                    noisetype="normal", tavg=tau_avg, coupling=coupling_eta, skp=skp_tag)
tser_tag, mser_tag, nuser_tag, cser_tag, cbarser_tag, thetaser_tag, wser_tag, bkvecser_tag = sim_results

### Check the output a bit

In [ ]:
skp = 50
fig, ax = plt.subplots()
w1_palette = sns.color_palette("Blues", n_colors=n_neurons)
w2_palette = sns.color_palette("Purples", n_colors=n_neurons)
w3_palette = sns.color_palette("Greens", n_colors=n_neurons)
for i in range(n_neurons-1):
    ax.plot(tser_tag[::skp], mser_tag[::skp, i, 0], color=w1_palette[i], alpha=0.8)
    ax.plot(tser_tag[::skp], mser_tag[::skp, i, 1], color=w2_palette[i], alpha=0.8)
    ax.plot(tser_tag[::skp], mser_tag[::skp, i, 2], color=w3_palette[i], alpha=0.8)
ax.plot(tser_tag[::skp], mser_tag[::skp, -1, 0], color=w1_palette[-1], label="Neuron Component 0", alpha=0.8)
ax.plot(tser_tag[::skp], mser_tag[::skp, -1, 1], color=w2_palette[-1], label="Neuron Component 1", alpha=0.8)
ax.plot(tser_tag[::skp], mser_tag[::skp, -1, 2], color=w3_palette[-1], label="Neuron Component 2", alpha=0.8)

ax.set(xlabel="Time", ylabel="Inhibition neurons components")
ax.legend()
plt.show()
plt.close()

### Compute and compare projection tags after inhibition

In [ ]:
### New odor, mix, and inhibit
### Repeat for many new odors (and ideally, should repeat for many backgrounds)
### But for now, assume all simulations would give similarly good inhibition. 
n_test_new_odors = 100
mix_frac = 0.5

# Average m and w with which we will inhibit
transient_tag = 100000 // skp_tag
mtag_mean = np.mean(mser_tag[transient_tag:], axis=0)
wtag_mean = np.mean(wser_tag[transient_tag:], axis=0)

# Background samples, valid for all new test odors
back_samples_tag = sample_background_thirdmoment(averages_nu, steady_covmat, epsilon_nu, back_components_tag, 
                                                  size=100, rgen=rgen_meta_tag)
inhib_ibcm_samples_tag = []
inhib_avg_samples_tag = []
mix_samples_tag = []
new_odor_targets = []
for i in range(n_test_new_odors):
    # New odor
    #new_odor_tag = np.roll(back_components_tag[0], shift=-1)  # Should be a new vector
    new_odor_tag = generate_odorant(n_dimensions_tag, rgen_meta_tag)
    new_odor_tag = new_odor_tag / l2_norm(new_odor_tag)
    new_odor_targets.append(new_odor_tag)

    mix_samples = back_samples_tag*(1.0 - mix_frac) + new_odor_tag.reshape(1, -1)*mix_frac
    mix_samples_tag.append(mix_samples)
    
    # Compare to inhibition of the average background
    avg_back_tag = averages_nu.dot(back_components_tag)
    a_over_ab = inhib_rates[0] / sum(inhib_rates)
    inhib_avg_samples = mix_samples - a_over_ab * avg_back_tag.reshape(1, -1)
    inhib_avg_samples_tag.append(inhib_avg_samples)

    # Inhibition of each generated sample and statistics on performance
    inhib_ibcm_samples = respond_new_odors(mix_samples, mtag_mean, wtag_mean, coupling_eta)
    inhib_ibcm_samples_tag.append(inhib_ibcm_samples)

mix_samples_tag = np.asarray(mix_samples_tag)
inhib_avg_samples_tag = np.asarray(inhib_avg_samples_tag)
inhib_ibcm_samples_tag = np.asarray(inhib_ibcm_samples_tag)
new_odor_targets = np.asarray(new_odor_targets)

In [ ]:
# Compute tags. This won't be great because too few dimensions to begin with, but try anyways. 
projtag_kwargs = dict(kc_sparsity=0.05, adapt_kc=True, n_pn_per_kc=6, fix_thresh=None)
proj_mat = create_sparse_proj_mat(n_kc=int(2000/50*n_dimensions_tag), n_rec=n_dimensions_tag, 
                        rgen=rgen_meta, fraction_filled=projtag_kwargs["n_pn_per_kc"]/n_dimensions_tag)

In [ ]:
# Compute tags and Jaccard distances between target odor and mixture without or with inhibition
jaccards_inhib_none = []
jaccards_inhib_avg = []
jaccards_inhib_ibcm = []
for i in range(mix_samples_tag.shape[0]):
    target_tag = project_neural_tag(new_odor_targets[i], new_odor_targets[i], proj_mat, **projtag_kwargs)
    for j in range(mix_samples_tag.shape[1]):
        tag_none = project_neural_tag(mix_samples_tag[i, j], mix_samples_tag[i, j], proj_mat, **projtag_kwargs)
        tag_avg = project_neural_tag(inhib_avg_samples_tag[i, j], mix_samples_tag[i, j], proj_mat, **projtag_kwargs)
        tag_ibcm = project_neural_tag(inhib_ibcm_samples_tag[i, j], mix_samples_tag[i, j], proj_mat, **projtag_kwargs)
        jaccards_inhib_none.append(jaccard(target_tag, tag_none))
        jaccards_inhib_avg.append(jaccard(target_tag, tag_avg))
        jaccards_inhib_ibcm.append(jaccard(target_tag, tag_ibcm))


In [ ]:
# Histograms of Jaccard similarities: larger similarity is better
fig, ax = plt.subplots()
clr_none = "xkcd:navy blue"
clr_ibcm = "xkcd:turquoise"
clr_avg = "xkcd:orangey brown"

ax.hist(jaccards_inhib_none, label="No inhibition", facecolor=clr_none, alpha=0.6, 
        edgecolor=clr_none, density=True)
ax.axvline(np.median(jaccards_inhib_none), color=clr_none, ls="--", lw=1.0)
ax.hist(jaccards_inhib_avg, label="Average inhibition", facecolor=clr_avg, alpha=0.6, 
        edgecolor=clr_avg, density=True)
ax.axvline(np.median(jaccards_inhib_avg), color=clr_avg, ls="--", lw=1.0)
ax.hist(jaccards_inhib_ibcm, label="IBCM inhibition", facecolor=clr_ibcm, alpha=0.6, 
        edgecolor=clr_ibcm, density=True)
ax.axvline(np.median(jaccards_inhib_ibcm), color=clr_ibcm, ls="--", lw=1.0)

ax.set(xlabel="Jaccard similarity", ylabel="Probability density", title="Jaccard similarity (higher is better)")
ax.legend()
fig.tight_layout()
#if mix_frac == 0.2:
#    fig.savefig("figures/detection/jaccard_similarity_ibcm_average_none_f20percent.pdf", transparent=True)
#elif mix_frac == 0.5:
#    fig.savefig("figures/detection/jaccard_similarity_ibcm_average_none_f50percent.pdf", transparent=True)
plt.show()
plt.close()

# Important potential issue
The way I inhibit the background based on the IBCM neurons' $\vec{m}$ and activity $c$ and the inhibitory neurons' averaged weights $\vec{w}$, that is, 

$$ \vec{s} = \vec{x} - \sum_i \overline{c}^i \vec{w}^i $$

is missing an important aspect: control of the inhibition intensity depending on a notion of "expected" fluctuations of the background. 
    
The IBCM neurons and inhibitory neurons are, together, learning ($\vec{w}$, $\vec{m}$) off-average components, and getting activated ($c$) when those components are more or less present in the background. Very well. 

But then, how much should those components be suppressed when they are present? The network should not just "blindly" suppress each component in its entirety. Rather, we want it to suppress only "typical" levels of those components, and let through "new" or "unexpected" levels of those components. 


So, what we want are more neurons that learn the typical distribution of the activation levels $c^i$ of the different pairs of neurons (sensitive to different components), and prevent excessive inhibition when the $c^i$ reach highly improbable values. 
 - One way to do this: would need to actually estimate the density of the joint distribution of the $c^i$, and stop inhibiting when the $c^i$ jump to low likelihood values. But that is hard and not necessarily so efficient. 
 - Another, simpler way to do this: learn limits (e.g. mean plus or minus stdev) on $c$, and clip the inhibition levels controlled by $c$ in $\vec{s}$. So, we should not just use , but some function of $c^i$ which is peaked at the mean and eventually decays to zero when $c$ goes too far off that mean. 
 
So, the idea is that, instead of pairs of IBCM and inhibitory neurons, we would have triplets: IBCM, inhibitory, gating. The "gating" neurons would have to learn a response function to $c$ which is like a gaussian. 

Is there a learning rule such that the output of a neuron is the "probability" of its input value? I'm sure there is, but unclear how to get that. 

I think the IBCM model may actually be able to do that. So we would need to put a second IBCM neuron that tracks the activity $c$ of the first and respond strongly to the mean $c$, and less and less strongly to off-average values. Is that the "coincidence detector" proposed by Intrator, 1997? Maybe! 

Read again this 1997 paper with this need of a system that learns "expected" fluctuations and lets through "novelties". 

### Summary of the inhibition model to explore:
Have many triplets of neurons, each triplet learning to inhibit a different component of the background appropriately with respect to the distribution of fluctuations of that background. The three neurons in a triplet and their role are:
 - "Component" IBCM neuron: takes as an input the ORN levels, becomes specific to an off-average component in the background. 
 - Inhibitory neuron: learn the background odor to which the "component", by averaging the ORN inputs weighted by the activity c of the "component" IBCM neuron (i.e. learning the average c^i*x). 
 - "Activity" IBCM neuron: takes as an input the activation c of the "component" IBCM neuron, and controls the extent at which the component is suppressed from the projection neurons, depending on its own output, call it p^i: s = x - p^i w^i.  
 
In prime, we could maybe get a novelty detection on an even slower time scale with one more IBCM neuron taking as a vector of input the activation level of all "component" IBCM neurons, and becoming specific to certain combinations of backgrounds. 

I'm not sure the "activity" neuron can be IBCM, but the goal is the same, just maybe need a different kind of neuron. 

# Comparison to ideal inhibitory network
In a linear algebra perspective, the best inhibition that could possibly be achieved of a new odor plus background mixture is that the whole component of the new odor parallel to the vector subspace spanned by the background odors is suppressed, while the component perpendicular to it is kept. Indeed, the appearance of the new odor's component in the background space cannot be distinguished from a fluctuation of the background (unless we had neurons tracking statistics of typical activations in that space, but not obvious how to get that). At any rate, this is the best we can hope our IBCM inhibition network will achieve. 

In [ ]:
def find_projector(a):
    """ Calculate projector a a^+, which projects
    a column vector on the vector space spanned by columns of a. 
    """
    a_inv = np.linalg.pinv(a)
    return a.dot(a_inv)
    
def find_parallel_component(x, basis, projector=None):
    """
    Args:
        x (np.ndarray): 1d array of length D containing the vector to decompose. 
        basis (np.ndarray): 2d matrix of size DxK where each column is one
            of the linearly independent background vectors. 
        projector (np.ndarray): 2d matrix A A^+, the projector on the vector
            space spanned by columns of basis. 
    Return:
        x_par (np.ndarray): component of x found in the vector space of basis
            The perpendicular component can be obtained as x - x_par. 
    """
    # If the projector is not provided yet
    if projector is None:
        # Compute Moore-Penrose pseudo-inverse and AA^+ projector
        projector = find_projector(basis)
    x_par = projector.dot(x)
    return x_par

def ideal_linear_inhibitor(x_n_par, x_n_ort, x_back, f, alpha, beta):
    """ Calculate the ideal projection neuron layer, which assumes
    perfect inhibition (down to beta/(alpha+beta)) of the component of the mixture
    parallel to the background odors' vector space, while leaving the orthogonal
    component of the new odor untouched. 
    
    Args:
        x_n_par (np.1darray): new odor, component parallel to background vector space
        x_n_ort (np.1darray): new odor, component orthogonal to background vector space 
        x_back (np.2darray): background samples, one per row
        f (float): mixture fraction (hard case is f=0.2)
        alpha (float): inhibitory weights learning rate alpha
        beta (float): inhibitory weights decaying rate beta
    
    Returns:
        s (np.1darray): projection neurons after perfect linear inhibition
    """
    # Allow broadcasting for multiple x_back vectors
    factor = beta / (alpha + beta)
    s = factor * f*x_n_par + f*x_n_ort
    # I thought the following would have been even better, but turns out it is worse for small f
    #s = f*x_n_par + f*x_n_ort
    s = s.reshape(1, -1) + factor * (1.0-f) * x_back
    return s

In [ ]:
# Reuse each new odor in new_odor_targets and each background in back_samples_tag
# Compute the projector on the background odor components only once
# Compute parallel component of each new odor
# Mix it with all background samples at once using broadcasting capability of ideal_linear_inhibitor function
background_projector = find_projector(back_components_tag.T)
inhib_ideal_samples_tag = []
for od in new_odor_targets:
    # Decompose
    od_par = find_parallel_component(od, basis=back_components_tag.T, projector=background_projector)
    od_ort = od - od_par
    # Compute the perfectly inhibited mixture with each background sample
    inhib_ideal = ideal_linear_inhibitor(od_par, od_ort, back_samples_tag, mix_frac, *inhib_rates)
    # Background reduced to b/(a+b), new odor intact? Perfect inhibition
    #inhib_ideal = inhib_rates[1] / sum(inhib_rates) * (1.0 - mix_frac) * back_samples_tag + mix_frac * od.reshape(1, -1)
    inhib_ideal_samples_tag.append(inhib_ideal)
inhib_ideal_samples_tag = np.asarray(inhib_ideal_samples_tag)

# Compute neural tags of the ideal inhibited mixtures and compare to target tags. 
# Compute tags and Jaccard distances between target odor and mixture without or with inhibition
jaccards_inhib_ideal = []
for i in range(new_odor_targets.shape[0]):
    target_tag = project_neural_tag(new_odor_targets[i], new_odor_targets[i], proj_mat, **projtag_kwargs)
    for j in range(back_samples_tag.shape[0]):
        mix_sample_tag = back_samples_tag[j] + mix_frac * new_odor_targets[i]
        tag_ideal = project_neural_tag(inhib_ideal_samples_tag[i, j], mix_sample_tag, proj_mat, **projtag_kwargs)
        jaccards_inhib_ideal.append(jaccard(target_tag, tag_ideal))


In [ ]:
# Histograms of Jaccard similarities: larger similarity is better
fig, ax = plt.subplots()
clr_map = {"none": "xkcd:navy blue", "average": "xkcd:orangey brown", 
           "ibcm":"xkcd:turquoise", "ideal": "xkcd:powder blue", "ideal2":"xkcd:pale rose"}

ax.hist(jaccards_inhib_ideal, label="Ideal inhibition", facecolor=clr_map["ideal"], alpha=0.6, 
        edgecolor=clr_map["ideal"], density=True)
ax.axvline(np.median(jaccards_inhib_ideal), color=clr_map["ideal"], ls="--", lw=1.0)
ax.hist(jaccards_inhib_avg, label="Average inhibition", facecolor=clr_map["average"], alpha=0.6, 
        edgecolor=clr_map["average"], density=True)
ax.axvline(np.median(jaccards_inhib_avg), color=clr_map["average"], ls="--", lw=1.0)
ax.hist(jaccards_inhib_ibcm, label="IBCM inhibition", facecolor=clr_map["ibcm"], alpha=0.6, 
        edgecolor=clr_map["ibcm"], density=True)
ax.axvline(np.median(jaccards_inhib_ibcm), color=clr_map["ibcm"], ls="--", lw=1.0)

ax.set(xlabel="Jaccard similarity", ylabel="Probability density", title="Jaccard similarity (higher is better)")
ax.legend()
fig.tight_layout()
plt.show()
plt.close()

# Performance as a function of f
I expect to see a relatively sharp drop of median performance for IBCM a bit above $f=\beta/(\alpha + \beta)$, and at this value for the ideal inhibition. 

In [ ]:
def compute_median_performances(back_samples, new_odors, f, projmat, proj_kwargs, 
                                m_mean, w_mean, eta, inhib_ab, back_components):
    """ Compute median Jaccard similarity for the different inhibition methods we have, 
    for a given value of mixture parameter f. """
    all_jaccard_pairs_dict = {"none":[], "average":[], "ibcm":[], "ideal":[], "ideal2":[]}  # list of lists, one per method
    back_proj = find_projector(back_components.T)
    for i in range(new_odors.shape[0]):
        # Compute target tag
        target_tag = project_neural_tag(new_odors[i], new_odors[i], projmat, **proj_kwargs)
        # Prepare mixtures
        mix_samples = back_samples*(1.0 - f) + new_odors[i:i+1]*f
        
        # Compute inhibited mixtures with the different methods
        # No inhibition: just use mix_samples
        # Average inhibition
        avg_back_tag = averages_nu.dot(back_components_tag)
        a_over_ab = inhib_ab[0] / sum(inhib_ab)
        inhib_avg_samples = mix_samples - a_over_ab * avg_back_tag.reshape(1, -1)

        # Inhibition of each generated sample and statistics on performance
        inhib_ibcm_samples = respond_new_odors(mix_samples, m_mean, w_mean, eta)
        
        # Ideal inhibition
        od_par = find_parallel_component(new_odors[i], basis=back_components.T, projector=back_proj)
        od_ort = new_odors[i] - od_par
        # Compute the perfectly inhibited mixture with each background sample
        inhib_ideal_samples = ideal_linear_inhibitor(od_par, od_ort, back_samples, f, *inhib_ab)
        # Background reduced to b/(a+b), new odor intact?
        inhib_ideal2_samples = (1.0 - a_over_ab) * (1.0 - f) * back_samples + f * new_odors[i:i+1]
    
        # For each inhibited mixture, compute jaccard similarity
        current_jaccard_dict = {"none":[], "average":[], "ibcm":[], "ideal":[], "ideal2":[]}
        for j in range(back_samples.shape[0]):
            mix_tag = project_neural_tag(mix_samples[j], mix_samples[j], projmat, **proj_kwargs)
            current_jaccard_dict["none"].append(jaccard(target_tag, mix_tag))
            
            # Average
            mix_tag = project_neural_tag(inhib_avg_samples[j], mix_samples[j], projmat, **proj_kwargs)
            current_jaccard_dict["average"].append(jaccard(target_tag, mix_tag))
        
            # IBCM
            mix_tag = project_neural_tag(inhib_ibcm_samples[j], mix_samples[j], projmat, **proj_kwargs)
            current_jaccard_dict["ibcm"].append(jaccard(target_tag, mix_tag))
            
            # Ideal
            mix_tag = project_neural_tag(inhib_ideal_samples[j], mix_samples[j], projmat, **proj_kwargs)
            current_jaccard_dict["ideal"].append(jaccard(target_tag, mix_tag))
            
            # Ideal 2
            mix_tag = project_neural_tag(inhib_ideal2_samples[j], mix_samples[j], projmat, **proj_kwargs)
            current_jaccard_dict["ideal2"].append(jaccard(target_tag, mix_tag))
            
        # Add those values to the total list
        for method in all_jaccard_pairs_dict.keys():
            all_jaccard_pairs_dict[method].append(current_jaccard_dict[method])
    
    # Convert to 2d array and compute median
    # Could choose to have one median per odor or per background sample
    all_jaccard_pairs_dict = {k:np.asarray(a) for k, a in all_jaccard_pairs_dict.items()}
    median_jaccard_pairs_dict = {k:np.median(a) for k, a in all_jaccard_pairs_dict.items()}
    return median_jaccard_pairs_dict
    

In [ ]:
# Use previous functions for various f values
median_jaccards = {"none":[], "average":[], "ibcm":[], "ideal":[], "ideal2":[]}
f_range = np.arange(0.1, 0.8, 0.1)
for f in f_range:
    meds = compute_median_performances(back_samples_tag, new_odor_targets, f, proj_mat, projtag_kwargs, 
                                mtag_mean, wtag_mean, coupling_eta, inhib_rates, back_components_tag)
    for k in meds:
        median_jaccards[k].append(meds[k])
    print("Done f = {:.2f}".format(f))

In [ ]:
fig, ax = plt.subplots()
labelmap = {"none":"None", "average":"Average", "ibcm":"IBCM", "ideal":r"Ideal $\perp$", "ideal2":"Ideal all"}
for k in median_jaccards:
    ax.plot(f_range, median_jaccards[k], color=clr_map[k], label=labelmap[k], lw=3)
ax.set(xlabel="Fraction $f$ of new odor", ylabel="Median Jaccard similarity")
ax.legend(title="Inhibition method")
fig.set_size_inches(4, 3)
fig.tight_layout()
#fig.savefig("figures/detection/inhibition_jaccard_comparison_methods.pdf", transparent=True)
plt.show()
plt.close()

## Analytical prediction for the "perfect" inhibition model
Perfect inhibition: only the background is reduced to $\frac{\beta}{\alpha + \beta}$ of its original amplitude, while the new odor is untouched:

$$ \vec{s} = \frac{\beta}{\alpha + \beta} (1-f) \vec{x}^b(t) + f \vec{x}^n $$

One would think this is the best possible inhibition, but for low $f$, it fares worse than deleting the parallel component of $\vec{x}^n$ as well, because inhibition of the parallel component lowers even further the activity of Kenyon cells specific to the background; the new odor does not reinforce them and only KCs specific to the new odor can cross the threshold. 

My analytical calculation is for this "perfect" inhibition, because it is easier to treat analytically than the . Also, I computed the mean rather than median Jaccard similarity. Anyways, let's see what it looks like and if it makes sense. 

UPDATE: It just does not work. 

In [ ]:
def analytical_approx_jaccard_perfect_inhibition_wrong(proj_kappa, avg_orn, avg_nu, k_back, 
                                                 kc_sparsity, f_mix, ab_rates):
    """
    This is not right at all, but I tried nevertheless. The average delta is causing
    a lot of issues. It seems one should really marginalize over deltas rather
    than replace it by some average delta value. 
    Args:
        proj_kappa (int): number of non-zero elements per row of the projection matrix
        avg_orn (float): average value of a component of an odor vector
        avg_nu (float): average value of the linear combination coefficients for t
            the background odors. 
        k_back (int): number of odors in the background
        kc_sparsity (float): fraction of KC cells kept in the tag (usually 0.05)
        f_mix (float): fraction of the mixture attributed to the new odor
        ab_rates (list of 2 floats): alpha, beta
    """
    # 1. Each KC activity, y_i, in response to the new odor follows gamma(proj_kappa, 1/avg_orn)
    # and in response to the average background, gamma(proj_kappa*k_back, 1/avg_orn/avg_nu)
    # 2.1 Calculate the threshold of most active KC cells in response to the new odor alone. 
    tau_new = sp.special.gammaincinv(proj_kappa, 1 - kc_sparsity) * avg_orn
    
    # 2.2 Calculate average margin above threshold, Delta, of the activities of the most active
    # KC in response to the new odor alone.
    # Try an approximation first: use some fraction of tau_new itself
    avg_delta = 0.05*tau_new
    # Otherwise, need to calculate complicated integral numerically
    
    # 3. Compute the probability, on average, that one of the KCs in the
    # new odor tag is in the tag of the background-new odor mixture
    lamb_nu = 1.0 / (avg_orn * avg_nu)
    factor = f_mix/(1 - f_mix) * sum(ab_rates)/ab_rates[1]
    avg_common_i = sp.special.gammaincc(proj_kappa*k_back, lamb_nu*(tau_new - factor * avg_delta))
    
    # 4. The average Jaccard similarity, using the inclusion-exclusion rule, is
    avg_jaccard = avg_common_i / (2.0 - avg_common_i)  # Ranges between 0 and 1, as it should
    
    return avg_jaccard

def analytical_approx_jaccard_perfect_inhibition(proj_kappa, avg_orn, avg_nu, k_back, 
                                                 kc_sparsity, f_mix, ab_rates):
    """
    Args:
        proj_kappa (int): number of non-zero elements per row of the projection matrix
        avg_orn (float): average value of a component of an odor vector
        avg_nu (float): average value of the linear combination coefficients for t
            the background odors. 
        k_back (int): number of odors in the background
        kc_sparsity (float): fraction of KC cells kept in the tag (usually 0.05)
        f_mix (float): fraction of the mixture attributed to the new odor
        ab_rates (list of 2 floats): alpha, beta
    """
    # 1. Each KC activity, y_i, in response to the new odor follows gamma(proj_kappa, 1/avg_orn),
    # and in response to the average background, gamma(proj_kappa*k_back, 1/avg_orn/avg_nu)
    # 2. Calculate the threshold of most active KC cells in response to the new odor alone. 
    tau_new = sp.special.gammaincinv(proj_kappa, 1 - kc_sparsity) * avg_orn
    
    # 3. Calculate the threshold of most active KC cells 
    # in response to the average background alone
    tau_b = sp.special.gammaincinv(proj_kappa*k_back, 1 - kc_sparsity) * (avg_orn * avg_nu)
    
    # The probability of a KC from the new odor's tag to be present in the mixture
    # tag as well is given by:
    #     avg_common_i = sp.special.gammaincc(proj_kappa*k_back, lamb_nu*(tau_new - factor * delta_i))
    # for a given value of delta_i. We need to average this over possible delta_i values
    # The prob distribution of delta_i is the gamma distribution, conditioned on knowing that y_i is larger
    # than tau_i, which is the (100*(1-kc_sparsity))-th percentile 
    # (so it is a gamma pdf(tau+delta) divided by kc_sparsity)
    loglambda_k = -proj_kappa * np.log(avg_orn)
    factor = f_mix/(1 - f_mix) * sum(ab_rates)/ab_rates[1]
    lamb_nu = 1.0 / (avg_orn * avg_nu)
    loggammak = sp.special.gammaln(proj_kappa)
    def integrand(delt):
        pdf_delta = (proj_kappa - 1)*np.log(tau_new + delt)  - (tau_new + delt)/avg_orn + loglambda_k
        pdf_delta = pdf_delta - loggammak
        cumul_dist_yib = sp.special.gammaincc(proj_kappa*k_back, max(0, lamb_nu*(tau_b - factor * delt)))
        return np.exp(pdf_delta) * cumul_dist_yib
    
    
    # Integrate (we kept the conditional prob. factor 1/kc_sparsity for the end)
    avg_common_i = sp.integrate.quad(integrand, 0, np.inf)[0]
    avg_common_i /= kc_sparsity
    
    # 4. The average Jaccard similarity, using the inclusion-exclusion rule, is
    avg_jaccard = avg_common_i / (2.0 - avg_common_i)  # Ranges between 0 and 1, as it should
    
    return avg_jaccard

In [ ]:
# Compute the analytical prediction for each f in f_range
predicted_perfect_jaccard = []
for f in f_range:
    predicted_perfect_jaccard.append(analytical_approx_jaccard_perfect_inhibition(
        proj_kappa=projtag_kwargs["n_pn_per_kc"], 
        avg_orn=np.mean(back_components_tag), 
        avg_nu=averages_nu.mean() + epsilon_nu*sigma2, 
        k_back=back_components_tag.shape[0],
        #avg_nu=averages_nu.mean()*0.36,  # Hack to get real distrib. of KC in response to background alone
        #k_back=back_components_tag.shape[0]/2.3,   # Hack
        kc_sparsity=projtag_kwargs["kc_sparsity"], 
        f_mix=f, ab_rates=inhib_rates)
    )
print(predicted_perfect_jaccard)

In [ ]:
fig, ax = plt.subplots()
labelmap = {"none":"None", "average":"Average", "ibcm":"IBCM", "ideal":r"Ideal $\perp$", "ideal2":"Ideal all"}
for k in ["ideal", "ideal2"]:
    ax.plot(f_range, median_jaccards[k], color=clr_map[k], label=labelmap[k], lw=3)
ax.plot(f_range, predicted_perfect_jaccard, color="k", label="Predicted all", lw=3)
ax.set(xlabel="Fraction $f$ of new odor", ylabel="Median Jaccard similarity")
ax.legend(title="Inhibition method")
fig.set_size_inches(4, 3)
fig.tight_layout()
#fig.savefig("figures/detection/ideal_inhibition_jaccard_vs_predicted.pdf", transparent=True)
plt.show()
plt.close()

In [ ]:
def log_gammadist(x, k, lam):
    return (k-1)*np.log(x) - lam*x + k*np.log(lam) - sp.special.gammaln(k)

In [ ]:
### Check distribution of KC in response to background, compare to the postulated gamma
# Dot product each back_sample_tag with the projmat, use all resulting KCs to determine distrib
# The correct kappa, K, avg_nu, etc. do not give the right distribution
# I can fit manually the gamma to get close to the actual distribution
# Try that hack in the calculation of the predicted performance
# UPDATE: does not seem to help...
back_samples_kenyon = proj_mat.dot(back_samples_tag.T)
kc_axis = np.linspace(back_samples_kenyon.min(), back_samples_kenyon.max(), 101)
# Compute gamma distribution that we postulated: gamma(K*kappa, lambda/avg_nu)
gamma_kc_axis = np.exp(log_gammadist(kc_axis, 
                            k=projtag_kwargs["n_pn_per_kc"]*back_components_tag.shape[0], 
                            lam=1.0 / (np.mean(back_components_tag) * averages_nu.mean())))
                            #k=projtag_kwargs["n_pn_per_kc"]*back_components_tag.shape[0]/2.3, 
                            #lam=0.36 / (np.mean(back_components_tag) * averages_nu.mean())))
plt.hist(back_samples_kenyon.flatten(), bins=20, density=True)
plt.plot(kc_axis, gamma_kc_axis)
plt.show()
plt.close()

In [ ]:
# Check distribution of KC in response to single new odor. Should really be a gamma distribution...
# This seems to match pretty closely. 
target_samples_kenyon = proj_mat.dot(new_odor_targets.T)
kc_axis = np.linspace(target_samples_kenyon.min(), target_samples_kenyon.max(), 101)
# Compute gamma distribution that we postulated: gamma(K*kappa, lambda/avg_nu)
gamma_kc_axis = np.exp(log_gammadist(kc_axis, 
                            k=projtag_kwargs["n_pn_per_kc"], 
                            lam=1.0 / (np.mean(new_odor_targets))))
plt.hist(target_samples_kenyon.flatten(), bins=20, density=True)
plt.plot(kc_axis, gamma_kc_axis)
plt.show()
plt.close()

# Investigate where IBCM fails
Is it just because of noise in  $\vec{m}$ and $\vec{w}$? Is it only some odors that fail terribly?

In [ ]:
back_samples_tag.shape